In [157]:
import os
import numpy as np

X = []
y = []
caracteristicas = []
classes = []

arquivos = os.listdir('../data/EMG_data_for_gestures-master/01')
#print(arquivos)

experimento_caracteristicas = [[],[],[],[],[],[]]
experimento_classes = []
    
for arquivo in arquivos:
    #le o arquivo e divide linha por linha
    conteudo = open('../data/EMG_data_for_gestures-master/01/' + arquivo).read().split('\n')
    #remove ultima linha (linha em branco)
    conteudo.pop()
    #remove primeira linha (cabecalho)
    conteudo.pop(0)
    #print(conteudo[0])
    
    for i in range(len(conteudo)):
        conteudo[i] = conteudo[i].split('\t')
        #remove primeira linha (campo "time")
        conteudo[i].pop(0)
        if conteudo[i][-1] != '0' and conteudo[i][-1] != '7':
            experimento_caracteristicas[int(conteudo[i][-1]) - 1].append(conteudo[i][:-1])
            
#print(len(experimento_caracteristicas))
#for i in range(len(experimento_caracteristicas)):
#    print(len(experimento_caracteristicas[i]))

menor = min([len(c) for c in experimento_caracteristicas])
#print('Menor:', menor)
caracs = [c[:menor] for c in experimento_caracteristicas]
caracs = np.array(caracs)
#print(caracs.shape)
caracs = caracs.swapaxes(1, 2)
caracs = caracs.astype(np.float)
print(caracs.shape)

(6, 8, 6806)


In [158]:
salto = 15
segmento = 32
n_win = int((caracs.shape[-1] - segmento) / salto) + 1
ids = np.arange(n_win) * salto
x = np.array([caracs[:,:,k:(k + segmento)] for k in ids]).transpose(1, 2, 0, 3)
x = x.transpose(0, 1, 3, 2)
x = x[:,:,:,:427]
x = x.transpose(0, 1, 3, 2)
print(x.shape)

(6, 8, 427, 32)


In [159]:
# VAR
var = np.sum(x ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1)
print(var.shape)
# RMS
rms = np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape[:-1]) - 1))
print(rms.shape)

(6, 8, 427)
(6, 8, 427)


In [160]:
from scipy.signal import stft

_, _, w = stft(caracs, fs=1000, nperseg=32, noverlap=16)
w = np.swapaxes(w, 2, 3)

print(w.shape)

(6, 8, 427, 17)


In [161]:
# definição da função PSD para o sinal no domínio da frequência
def PSD(x):
    return np.sqrt(np.abs(x))

# FMD
fmd = np.sum(PSD(w), axis=-1) / 2
print(fmd.shape)
# MMDF
mmdf = np.sum(np.abs(w), axis=-1) / 2
print(mmdf.shape)

(6, 8, 427)
(6, 8, 427)


In [162]:
#vetor de caracteristicas
features = list()
for feature in (var, rms, fmd, mmdf):
    feature = feature.transpose(0, 2, 1)
    feature = feature.reshape(6 * 849, 8)
    #print('Feature: {}'.format(feature), feature.shape)
    features.append(feature)

X = np.concatenate(features, axis=-1)
print(X.shape)

ValueError: cannot reshape array of size 20496 into shape (5094,8)

In [ ]:
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# dividindo as porções de dados em treino e teste (70 e 30% respectivamente)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

# o trabalho das diferentes parametrizações ficam a cargo dos alunos
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

In [ ]:
res = clf.predict(X_test)
tot_hit = sum([1 for i in range(len(res)) if res[i] == y_test[i]])
print('Acurácia: {:.2f}%'.format(tot_hit / X_test.shape[0] * 100))